# Retrieval-Augmented Generation

## Einführung

Im folgenden wird eine Retrieval-Augmented Generation (RAG) für die Magpie gebaut. Ziel ist es, einen Chatbot zu genieren, der Fragen in natürlicher Sprache aufnimmt und diese in passende SQL-Abfragen umwandelt. Diese werden wiederum dem hinter dem Chatbot stehenden LMM als Kontext übermittelt, sodass dieser wiederum in natürlicher Sprache antworten kann. 

Um die SQL Anfragen passgenau auf die Daten der Magpie anzupassen, wird weiterhin ein retriever tool erzeugt und hier verwendet. Die Erzeugung des retriever tools kann hier nachvollzogen werden: #TODO Link einfügen!

In einem ersten Schritt ermitteln wir unser Arbeitsverzeichnis und definieren — falls nötig — unser Stammverzeichnis.

In [2]:
import os

# Aktuelles Arbeitsverzeichnis ermitteln
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

## Kleine Exploration der Magpie

Wir laden weiterhin die Magpie und stellen eine Verbindung zu ihr her. Es wird `cursor` vom Verbindungsobjekt `conn` erstellt.
Ein `cursor` wird verwendet, um SQL-Abfragen an die Datenbank zu senden und Ergebnisse zurückzugeben.

In [3]:
import duckdb

conn = duckdb.connect("data/magpie.db")
cursor = conn.cursor()

Da pandas data frames wunderbar über den VS Code eigenen Data Viewer exploriert werden können, wandeln wir die Daten aus der Magpie in einen pandas data frame um. Für diese exploration wählen wir den Datensatz `datensatz_drittmittel_hochschule`.

In [4]:
import pandas as pd

query = "SELECT * FROM datensatz_drittmittel_hochschule;"
df = pd.read_sql(query, conn)
conn.close()

C:\Users\Hueck\AppData\Local\Temp\ipykernel_26432\262543780.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Erstellung des Chatbot
### Auswahl des LLMs

In einem ersten Schritt laden wir über das Repository [Ollama](https://ollama.com/) das Large Language Modell (LLM) `llama3.1`. Ollama ermöglicht es, LLMs lokal und ohne API d.h. kostenfrei zu verwenden.

In [5]:
from langchain_ollama import ChatOllama
import re

llm = ChatOllama(
    model="llama3.1:8b-instruct-q4_0",
    temperature=0,
    server_url="http://127.0.0.1:11434",
)

### Verbindung zur Magpie 

Im Folgenden wird zunächst die `SQLDatabase`-Klasse aus dem Modul `langchain_community.utilities` importiert. Anschließend wird mit `SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")` eine Verbindung zur DuckDB-Datenbank namens `drittmittel_hs.db` im Verzeichnis `data` aufgebaut. `drittmittel_hs.db` ist eine auf verjüngte Version der Magpie, die lediglich Daten zu Drittmitteln an Hochschulen beinhaltet.

Im Gegensatz zu dem obigen Zugriff auf die DuckDB mittels der nativen `duckdb`-Bibliothek, erfolgt der Zugriff nun über das `SQLDatabase`-Utility aus LangChain. Diese Methode abstrahiert viele Details und erleichtert die Integration von Datenbankabfragen in KI-gestützte Workflows. Während der direkte Zugriff ideal für individuelle SQL-Abfragen und maximale Flexibilität ist, eignet sich LangChain besonders für Anwendungen, bei denen Datenbankzugriffe in NLP- oder KI-Prozesse eingebettet werden sollen.


In [6]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")

c:\Users\Hueck\miniconda3\envs\RAG_LLM\lib\site-packages\duckdb_engine\__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


### Funktion Nr.1: `get_sql_chain`

#### Ziel
Die Funktion `get_sql_chain` erstellt eine SQL-Abfragekette, die auf Basis von natürlicher Sprache SQL-Abfragen generiert und ausführt. 

#### Parameter
1. `llm` (Language Model): Ein KI-Modell, das natürliche Spracheingaben interpretiert und SQL-Abfragen erstellt.
2. `db` (Datenbankverbindung): Die Datenbank, mit der die generierten SQL-Abfragen ausgeführt werden.
3. `table_info` (Tabellenschema): Ein String, der die Struktur und Details der Tabellen in der Datenbank beschreibt (z. B. Tabellen- und Spaltennamen).
4. `top_k` (Standardwert: 10): Die maximale Anzahl der zurückzugebenden Zeilen.

#### Funktionsweise der `get_sql_chain`-Funktion

1. In einem ersten Schritt wird das Objekt `template` erstellt, das klare Anweisungen für das Sprachmodell (LLM) bereitstellt. `template` dient als Anleitung, wie das Modell SQL-Abfragen basierend auf der Benutzeranfrage erstellen soll. 

2. Basierend auf der Vorlage `template` wird weiterhin ein `PromptTemplate` erstellt.Der `PromptTemplate` übernimmt Platzhalter (wie `{{table_info}}` und `{{input}}`), die später dynamisch mit echten Werten ersetzt werden. Dadurch wird ein flexibles Format erzeugt, das an das Sprachmodell gesendet werden kann.

3. Schließlich wird eine SQL-Abfragekette (`sql_chain`) erstellt, die folgende Komponenten kombiniert:<br>
   3.1 Das Sprachmodell (`llm`): Dieses Modell interpretiert die Benutzerfrage und erstellt eine SQL-Abfrage basierend auf der Vorlage.<br>
   3.2 Die Datenbankverbindung (`db`): Die generierte SQL-Abfrage wird ausgeführt, um Ergebnisse aus der Datenbank zu extrahieren.<br>
   3.3 Das Prompt-Template (`prompt`): Die Vorlage sorgt dafür, dass das Modell alle nötigen Informationen (z. B. Tabellenstruktur) hat, um präzise Abfragen zu erstellen.<br>


In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_sql_query_chain

def get_sql_chain(llm, db, table_info, top_k=10):
    
    template = f"""Given an input question, first create a syntactically
    correct SQL query to run in {db.dialect}, then look at the results of the
    query and return the answer to the input question. You can order the
    results to return the most informative data in the database.
    
    Unless otherwise specified, do not return more than {{top_k}} rows.

    Never query for all columns from a table. You must query only the
    columns that are needed to answer the question. Wrap each column name
    in double quotes (") to denote them as delimited identifiers.

    Pay attention to use only the column names present in the tables
    below. Be careful to not query for columns that do not exist. Also, pay
    attention to which column is in which table. Query only the columns you
    need to answer the question.
    
    Please carefully think before you answer.

    Here is the schema for the database:
    {{table_info}}

    Additional info: {{input}}

    Return only the SQL query such that your response could be copied
    verbatim into the SQL terminal.
    
    """

    prompt = PromptTemplate.from_template(template)

    sql_chain = create_sql_query_chain(llm, db, prompt)

    return sql_chain

### Funktion Nr. 2: `natural_language_chain`

#### Ziel
Die Funktion `natural_language_chain` verarbeitet eine Anfrage in natürlicher Sprache, generiert eine SQL-Abfrage basierend auf dieser Frage und der Datenbankstruktur, führt die Abfrage aus und gibt eine Antwort in natürlicher Sprache zurück.

#### Parameter
1. `question` (String): Die Frage des Benutzers in natürlicher Sprache.
2. `llm` (Language Model): Ein KI-Modell, das Sprache versteht und SQL-Abfragen erstellen kann.
3. `db` (Datenbankverbindung): Die Verbindung zur Datenbank, die die relevanten Daten enthält.

#### Funktionsweise der `natural_language_chain`

1. Tabelleninformationen abrufen  
   Die Tabellenstruktur der Datenbank wird mit `db.get_table_info()` abgerufen. Dies liefert Details zu Tabellen und Spalten.

2. SQL-Kette erstellen  
   Eine SQL-Abfragekette (`sql_chain`) wird mithilfe der Funktion `get_sql_chain` generiert. Diese nutzt die Tabelleninformationen und das Sprachmodell, um die Frage in eine SQL-Abfrage zu übersetzen.

3. Vorlage (Prompt) definieren  
   Eine Textvorlage wird erstellt, die bestimmt, wie die SQL-Abfrage, die Benutzerfrage und die SQL-Antwort in eine verständliche Antwort umgewandelt werden. Die Vorlage enthält Platzhalter für:
   - Die SQL-Abfrage (`{{query}}`)
   - Die Benutzerfrage (`{{question}}`)
   - Die SQL-Antwort (`{{response}}`)

4. Zwischenschritt zur SQL-Generierung  
   - Ein Zwischenprozess wird definiert, um die SQL-Abfrage aus der Benutzerfrage zu extrahieren (`RunnablePassthrough.assign`).
   - Das generierte SQL wird für Debugging-Zwecke ausgegeben.

5. Vollständige Kette ausführen  
   - Eine Verarbeitungssequenz wird erstellt, die:
     - Die SQL-Abfrage ausführt.
     - Die Datenbankantwort abruft.
     - Basierend auf dem Ergebnis eine Antwort erstellt, die auf die Benutzerfrage zugeschnitten ist.

6. Antwort generieren  
   - Die Antwort wird durch die vollständige Kette generiert und zurückgegeben. Sie ist in natürlicher Sprache und enthält die relevanten Informationen aus der Datenbank.


In [28]:
def natural_language_chain(question, llm, db):
    table_info = db.get_table_info()
    sql_chain = get_sql_chain(llm, db, table_info=table_info)

    template = f"""
        You are a chatbot named >>Sparklehorse<< created by the 
        >>Stifterverband für die Deutsche Wissenschaft<<. Based on the table schema given below, the SQL query and the SQL response, enter an answer
        that corresponds to the language of the user's question. Think carefully and make sure that your answer is precise and easy to understand.

        SQL Query: {{query}}
        User question: {{question}}
        SQL Response: {{response}}

        Guidelines:
        - Only answer questions related to the database.
        - If unsure, default to "I can only answer questions related to the database."
        """

    prompt = PromptTemplate.from_template(template)

    # Create the intermediate chain to extract SQL query
    intermediate_chain = RunnablePassthrough.assign(query=sql_chain)

    # Get the SQL query
    intermediate_result = intermediate_chain.invoke({"question": question})
    sql_query = intermediate_result["query"]

    # Debug: Print the SQL query
    print("Generated SQL Query for Debugging:")
    print(sql_query)

    # Continue with the full chain execution
    chain = (
        intermediate_chain.assign(
            response=itemgetter("query") | QuerySQLDataBaseTool(db=db)
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    response = chain.invoke({"question": question})

    print(response)

    return response

In [26]:
_ = natural_language_chain('Wie hoch waren die "Drittmittel von Gemeinden und Zweckverbänden" der Universität Kassel im Jahr 2008?', llm, db)

Generated SQL Query for Debugging:
SELECT "Wert"
FROM datensatz_drittmittel_hochschule
WHERE "Variable" = 'Drittmittel von Gemeinden und Zweckverbänden' AND "Hochschule" = 'Universität Kassel' AND jahr = 2008;
Hallo!

Die Antwort auf Ihre Frage ist einfach: Die Drittmittel von Gemeinden und Zweckverbänden der Universität Kassel im Jahr 2008 betrugen 107.659,99 Euro.

Keine weitere Information erforderlich!


In [29]:
_ = natural_language_chain('Wer ist Werner Herzog?', llm, db)


Generated SQL Query for Debugging:
SELECT "Hochschule", "Wert"
FROM datensatz_drittmittel_hochschule
WHERE "Variable" = 'Drittmittel vom Bund'
ORDER BY "Jahr" DESC LIMIT 5;
A nice change of pace from the usual technical queries!

Since the user's question is not related to the database, I'll respond accordingly:

"I'm afraid I can only answer questions related to the database. The query you provided and the table schema suggest a focus on higher education funding data. Unfortunately, Werner Herzog, the famous German film director, doesn't seem to be connected to this dataset."
